## Basic exploratory analysis on the parquet files

### Use pyarrow and pandas to read the parquet files to understand what kind of information we're getting

In [1]:
import pandas as pd
import pyarrow.dataset as ds

In [2]:
dataset = ds.dataset("../data/input", format="parquet")
df = dataset.to_table().to_pandas()

In [3]:
df.head()

,event_time,user_id,account_id,amount,transaction_type,cdc_operation,cdc_sequence_num,source_system
0,2024-08-03 04:52:37.668,0800b8e0-c3f5-5fe8-bcbc-1fcbb35ca9d0,eefbde26-aecf-5fff-a516-7ceb79b36bf1,-137.66,WITHDRAWAL,insert,2982801,wallet_service
1,2024-08-03 04:52:40.686,4fd493ab-b3a8-595e-8b58-c46c8ff971e5,55af41da-ab21-5677-9342-4fe065e2ce2a,-144.68,WITHDRAWAL,insert,2982802,wallet_service
2,2024-08-03 04:52:43.838,ec573bb7-807e-573d-a0e2-07e19de1c034,6e7a4987-a3ca-552f-9cfa-1eefb7de4cf2,-65.67,TRANSFER_OUT,insert,2982803,wallet_service
3,2024-08-03 04:52:48.265,e061df1c-75df-56d7-b7de-5bd698033aab,2d60329f-624c-5a93-964c-6faef2b734bb,152.91,TRANSFER_IN,insert,2982804,wallet_service
4,2024-08-03 04:52:50.621,97bf868b-3748-5aa8-933c-5bf72705aec4,49f6bdd0-6b70-5d1f-97a8-77c27a937dd8,861.01,TRANSFER_IN,insert,2982805,wallet_service


### Running some checks on the data

In [4]:
# Row count
row_count = len(df)
print(f"Total rows: {row_count}")

Total rows: 5023665


In [5]:
# Null value check
null_counts = df.isnull().sum()
print("Nulls per column:")
print(null_counts)

Nulls per column:
event_time          0
user_id             0
account_id          0
amount              0
transaction_type    0
cdc_operation       0
cdc_sequence_num    0
source_system       0
dtype: int64


In [6]:
# transaction_type
tx_counts = df.groupby("transaction_type").size().reset_index(name="count")
print("Counts by transaction_type:")
print(tx_counts)

Counts by transaction_type:
  transaction_type    count
0          DEPOSIT   841880
1      TRANSFER_IN  1710624
2     TRANSFER_OUT  1633064
3   WALLET_CREATED    23665
4       WITHDRAWAL   814432


In [7]:
# cdc_operation
cdc_counts = df.groupby("cdc_operation").size().reset_index(name="count")
print("Counts by cdc_operation:")
print(cdc_counts)

Counts by cdc_operation:
  cdc_operation    count
0        insert  5023665


In [8]:
# source_system
src_counts = df.groupby("source_system").size().reset_index(name="count")
print("Counts by source_system:")
print(src_counts)

Counts by source_system:
    source_system    count
0  wallet_service  5023665


In [9]:
# format the timestamp field
df['event_time'] = pd.to_datetime(df['event_time'])
df['event_date'] = df['event_time'].dt.date

In [10]:
# group by that date and count
daily_counts = (
    df
    .groupby('event_date')
    .size()
    .reset_index(name='count')
    .sort_values('event_date')
)
print(daily_counts)

     event_date  count
0    2024-05-01  39746
1    2024-05-02  33577
2    2024-05-03  32794
3    2024-05-04  32105
4    2024-05-05  32213
..          ...    ...
155  2024-10-03  31514
156  2024-10-04  31344
157  2024-10-05  31451
158  2024-10-06  31541
159  2024-10-07   2632

[160 rows x 2 columns]


In [11]:
# List all the fields from the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5023665 entries, 0 to 5023664
Data columns (total 9 columns):
 #   Column            Dtype         
---  ------            -----         
 0   event_time        datetime64[ns]
 1   user_id           object        
 2   account_id        object        
 3   amount            object        
 4   transaction_type  object        
 5   cdc_operation     object        
 6   cdc_sequence_num  int64         
 7   source_system     object        
 8   event_date        object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 344.9+ MB
